### Entrenamiento de modelo XGboost 
#####Autor: Unidad de Ciencia de Datos (UCD)

El presente cuaderno muestra los pasos a seguir para entrenar el modelo XGboost elaborado en el piloto DataSandbox Colombia. 

La estructura del cuaderno consta de 3 etapas:
1. Importación de paquetes y conjuntos de datos necesarios para el análisis.
2. Definición del modelo de clasificación XGboost. 
3. Proceso de entrenamiento y predicción.

##### 1. Importación de paquetes y conjuntos de datos necesarios para el análisis.

In [0]:
'''Antes de importar los paquetes instale las siguientes librerías:

    numpay version 1.18.1
    pandas version 1.0.1
    mlflow version 1.11.0
    xgboost version 1.2.0 
    scikit-learn version 0.22.1 

''' 
import numpy as np
from numpy import arange, argmax
import pandas as pd
import math
from math import exp

import mlflow                                                              
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient
import mlflow.xgboost

import xgboost as xgb                                                    
from sklearn.metrics import roc_auc_score, f1_score, make_scorer

import hyperopt
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK, rand
from hyperopt.pyll import scope

Para replicar el proceso de clasificación de imágenes satelitales es necesario que importe los conjuntos de [datos](https://github.com/ucd-dnp/inudaciones_ucd/tree/master/dataSandbox/3_Datos_entrenamiento) *Train* y *Test* de la carpeta del Piloto DataSandbox Colombia que puede encontrar en el [repositorio](https://github.com/ucd-dnp/inudaciones_ucd/tree/master/dataSandbox) de Github. Estos conjuntos de datos surgen de una división aleatoria con proporción 0.85 correspondiente al conjunto de entrenamiento y 0.15 correspondiente al conjunto de prueba.

In [0]:
test = pd.read_csv("/dbfs/FileStore/shared_uploads/aburitica@dnp.gov.co/Test.csv")
train = pd.read_csv("/dbfs/FileStore/shared_uploads/aburitica@dnp.gov.co/Train-1.csv")

#A continuación se dividen los datos de entrenamiento y prueba en X(conjunto de atributos) e Y(conjunto de etiquetas).
X_train = train.drop(['construc'], axis = 1)
X_test = test.drop(['construc'], axis = 1)
y_train = train.construc.astype(int)
y_test = test.construc.astype(int)

##### 2. Definición del modelo de clasificación XGboost.

A continuacion se define el conjunto de parámetros deseado del modelo XGboost y la función con la cual se ejecuta el entrenamiento del modelo.

In [0]:
params = {
  'max_depth': 19,
  'learning_rate': 0.0695596075784221, 
  'reg_alpha': 0.341448738908714,
  'reg_lambda': 0.155133834122141,
  'min_child_weight': 11.995066302931669,
  'objective': 'binary:logistic',
  'seed': 123, # Establezca una semilla para el que el entrenamiento sea replicable
}

In [0]:
def train_model(params):
  '''
  Esta función ejecuta el entrenamiento del modelo XGboost tomando como insumo el conjunto de parámetros deseado. 
  '''
  #hyperparameters and the trained model to MLflow.
  mlflow.xgboost.autolog()
  with mlflow.start_run(nested=True):
    
    train = xgb.DMatrix(data=X_train, label=y_train)
    test = xgb.DMatrix(data=X_test, label=y_test)
    
    booster = xgb.train(params=params,
                        dtrain=train,
                        num_boost_round=1000,\
                        evals=[(test, "test")],
                        early_stopping_rounds=50)
    
    predictions_test = booster.predict(test)
    auc_score = roc_auc_score(y_test, predictions_test)
    f1 = f1_score(y_test, predictions_test.round())
    mlflow.log_metric('auc', auc_score)
    mlflow.log_metric('f1', f1)
 
    signature = infer_signature(X_train, booster.predict(train))
    mlflow.xgboost.log_model(booster,
                             "model",
                             signature=signature)
    
    #los:-1*f1 so fmin maximizes the f1
    return {'status': STATUS_OK, 'loss': -1*f1, 'booster': booster.attributes()}

Por último, ejecute la función definida y registre el modelo usando la API de seguimiento de MLflow. Para mayor información puede consultar el siguiente [tutorial](https://docs.microsoft.com/en-us/azure/databricks/applications/mlflow/tracking#tracking-machine-learning-training-runs).

In [0]:
train_model(params)

/databricks/python/lib/python3.7/site-packages/mlflow/xgboost.py:364: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
 all_arg_names = inspect.getargspec(original)[0] # pylint: disable=W1505
[0]	test-error:0.17672
Will train until test-error hasn't improved in 50 rounds.
[1]	test-error:0.17191
[2]	test-error:0.17078
[3]	test-error:0.16689
[4]	test-error:0.16474
[5]	test-error:0.16474
[6]	test-error:0.16310
[7]	test-error:0.16197
[8]	test-error:0.16238
[9]	test-error:0.16033
[10]	test-error:0.16003
[11]	test-error:0.15921
[12]	test-error:0.15787
[13]	test-error:0.15705
[14]	test-error:0.15664
[15]	test-error:0.15541
[16]	test-error:0.15531
[17]	test-error:0.15490
[18]	test-error:0.15460
[19]	test-error:0.15490
[20]	test-error:0.15470
[21]	test-error:0.15501
[22]	test-error:0.15439
[23]	test-error:0.15398
[24]	test-error:0.15326
[25]	test-error:0.15296
[26]	test-error:0.15214
[27]	test-error:0.15121
[28]	test-error:0.15152
[29]	test-error:0.15060
[30]	test-error:0.15070
[31]	test-error:0.15050
[32]	test-error:0.14916
[33]	test-error:0.14855
[34]	test-error:0.14814
[35]	test-error:0.14845
[36]	test-error:0.14855
[37]	test-error:0.14794
[38]	test-error:0.14722
[39]	test-error:0.14599
[40]	test-error:0.14589
[41]	test-error:0.14558
[42]	test-error:0.14578
[43]	test-error:0.14537
[44]	test-error:0.14445
[45]	test-error:0.14455
[46]	test-error:0.14507
[47]	test-error:0.14548
[48]	test-error:0.14568
[49]	test-error:0.14568
[50]	test-error:0.14589
[51]	test-error:0.14599
[52]	test-error:0.14640
[53]	test-error:0.14507
[54]	test-error:0.14548
[55]	test-error:0.14517
[56]	test-error:0.14497
[57]	test-error:0.14537
[58]	test-error:0.14425
[59]	test-error:0.14476
[60]	test-error:0.14486
[61]	test-error:0.14507
[62]	test-error:0.14435
[63]	test-error:0.14363
[64]	test-error:0.14333
[65]	test-error:0.14312
[66]	test-error:0.14302
[67]	test-error:0.14312
[68]	test-error:0.14322
[69]	test-error:0.14302
[70]	test-error:0.14251
[71]	test-error:0.14281
[72]	test-error:0.14251
[73]	test-error:0.14210
[74]	test-error:0.14148
[75]	test-error:0.14128
[76]	test-error:0.14138
[77]	test-error:0.14107
[78]	test-error:0.14189
[79]	test-error:0.14189
[80]	test-error:0.14230
[81]	test-error:0.14240
[82]	test-error:0.14251
[83]	test-error:0.14189
[84]	test-error:0.14158
[85]	test-error:0.14158
[86]	test-error:0.14128
[87]	test-error:0.14087
[88]	test-error:0.14117
[89]	test-error:0.14169
[90]	test-error:0.14179
[91]	test-error:0.14179
[92]	test-error:0.14240
[93]	test-error:0.14230
[94]	test-error:0.14210
[95]	test-error:0.14158
[96]	test-error:0.14169
[97]	test-error:0.14138
[98]	test-error:0.14169
[99]	test-error:0.14199
[100]	test-error:0.14169
[101]	test-error:0.14169
[102]	test-error:0.14240
[103]	test-error:0.14189
[104]	test-error:0.14210
[105]	test-error:0.14179
[106]	test-error:0.14169
[107]	test-error:0.14128
[108]	test-error:0.14066
[109]	test-error:0.14035
[110]	test-error:0.13984
[111]	test-error:0.14015
[112]	test-error:0.14035
[113]	test-error:0.14035
[114]	test-error:0.14046
[115]	test-error:0.14035
[116]	test-error:0.14056
[117]	test-error:0.14066
[118]	test-error:0.14046
[119]	test-error:0.14066
[120]	test-error:0.14107
[121]	test-error:0.14128
[122]	test-error:0.14128
[123]	test-error:0.14076
[124]	test-error:0.14005
[125]	test-error:0.14005
[126]	test-error:0.14056
[127]	test-error:0.13964
[128]	test-error:0.13953
[129]	test-error:0.14056
[130]	test-error:0.14046
[131]	test-error:0.14066
[132]	test-error:0.14066
[133]	test-error:0.14046
[134]	test-error:0.14056
[135]	test-error:0.14025
[136]	test-error:0.14087
[137]	test-error:0.14107
[138]	test-error:0.14015
[139]	test-error:0.14025
[140]	test-error:0.14035
[141]	test-error:0.13984
[142]	test-error:0.14005
[143]	test-error:0.13995
[144]	test-error:0.14025
[145]	test-error:0.14025
[146]	test-error:0.14035
[147]	test-error:0.14035
[148]	test-error:0.14015
[149]	test-error:0.13912
[150]	test-error:0.14046
[151]	tes